In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from tqdm import tqdm
from train import ConditionalUNet  # Import trained DDPM model
from load_dataset import get_dataloader  # Load dataset



# Load trained classifiers
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Function to convert a list of label dictionaries into binary class tensors
def convert_labels_2(true_labels, attribute_name):
    # Ensure everything is moved to the correct device
    device = true_labels[0]["size"].device

    # Initialize empty lists for merging
    merged_labels = {
        "shape": [],
        "color": [],
        "size": [],
        "bg_color": []
    }

    # Iterate over the batch dictionaries and append all samples to the merged list
    for batch in true_labels:
        merged_labels["shape"].extend(batch["shape"])  # This is already a list of strings
        merged_labels["color"].append(batch["color"])  # Append tensors
        merged_labels["size"].append(batch["size"])    # Append tensors
        merged_labels["bg_color"].append(batch["bg_color"])  # Append tensors

    # Stack tensors for batch processing
    merged_labels["color"] = torch.cat(merged_labels["color"]).to(device)  # Shape: (total_batch_size, 3)
    merged_labels["size"] = torch.cat(merged_labels["size"]).to(device)  # Shape: (total_batch_size,)
    merged_labels["bg_color"] = torch.cat(merged_labels["bg_color"]).to(device)  # Shape: (total_batch_size, 3)

    # Convert each attribute into a binary tensor with correct shape (batch_size,)
    if attribute_name == "shape":
        return torch.tensor([0 if shape == "circle" else 1 for shape in merged_labels["shape"]], dtype=torch.long, device=device)

    elif attribute_name == "color":
        color_class = (merged_labels["color"][:, 2] > merged_labels["color"][:, 0]).long() # 0 for red, 1 for blue
        return color_class.view(-1)  # Ensure shape is (batch_size,)

    elif attribute_name == "size":
        size_class = (merged_labels["size"] >= 0.5).long()  # 0 for small, 1 for large
        return size_class.view(-1)  # Ensure shape is (batch_size,)

    elif attribute_name == "bg_color":    
        brightness = merged_labels["bg_color"].mean(dim=1)
        bg_class = (brightness >= 0.5).long()  # 0 for dark, 1 for bright
        return bg_class.view(-1)  # Ensure shape is (batch_size,)


    else:
        raise ValueError(f"Unknown attribute name: {attribute_name}")


In [2]:

# Define the classifier structure (must match training)
class AttributeClassifier(nn.Module):
    def __init__(self, input_dim, num_classes=2):
        super(AttributeClassifier, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        return self.fc(x)

# Function to load a classifier
def load_classifier(model_path):
    classifier = AttributeClassifier(input_dim=3*32*32, num_classes=2).to(device)  # Flattened image input
    classifier.load_state_dict(torch.load(model_path, map_location=device))
    classifier.eval()  # Set to evaluation mode
    return classifier

In [3]:
# Load all classifiers
shape_classifier = load_classifier("models/shape_classifier.pth")
color_classifier = load_classifier("models/color_classifier.pth")
size_classifier = load_classifier("models/size_classifier.pth")
bg_color_classifier = load_classifier("models/bg_color_classifier.pth")

print("All classifiers loaded successfully!")


# Load trained DDPM model
ddpm_model = ConditionalUNet().to(device)
ddpm_model.load_state_dict(torch.load("models/ddpm_conditional_unet.pth", map_location=device))
ddpm_model.eval()
print("DDPM model loaded successfully!")

All classifiers loaded successfully!
DDPM model loaded successfully!


/tmp/ipykernel_388526/3115439719.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  classifier.load_state_dict(torch.load(model_path, map_location=device))
/tmp/ipykernel_

In [4]:
# Function to generate images using the DDPM model
def generate_images(ddpm_model, dataset_loader, n_T=1000):
    ddpm_model.eval()
    generated_images = []
    true_labels = []

    with torch.no_grad():
        for images, labels in tqdm(dataset_loader, desc="Generating Images from DDPM"):
            images = images.to(device)

            # Extract conditioning information
            conditions = torch.stack([
                labels["x_position"],
                labels["y_position"],
                labels["color"][:, 0],
                labels["color"][:, 1],
                labels["color"][:, 2],
                labels["size"],
                labels["bg_color"][:, 0],
                labels["bg_color"][:, 1],
                labels["bg_color"][:, 2],
                labels["class"].float()
            ], dim=1).to(device)

            # Start from pure noise and reverse diffusion
            x_t = torch.randn_like(images).to(device)
            for t in reversed(range(n_T)):
                predicted_noise = ddpm_model(x_t, conditions)
                x_t = x_t - predicted_noise  # Reverse diffusion step

            generated_images.append(x_t.cpu())  # Store generated image
            true_labels.append(labels)  # Store ground truth labels

    return torch.cat(generated_images), true_labels


In [5]:
# Load test dataset
test_loader = get_dataloader(batch_size=8, training=False)

# Generate images
generated_images, true_labels = generate_images(ddpm_model, test_loader)

Loaded 128 testing images.


Generating Images from DDPM: 100%|██████████| 16/16 [00:40<00:00,  2.52s/it]


In [6]:
generated_images = generated_images.view(generated_images.shape[0], -1).to(device)

In [7]:
pred_shape = torch.argmax(shape_classifier(generated_images), dim=1)
pred_color = torch.argmax(color_classifier(generated_images), dim=1)
pred_size = torch.argmax(size_classifier(generated_images), dim=1)
pred_bg = torch.argmax(bg_color_classifier(generated_images), dim=1)

In [8]:
print(pred_shape)
print(pred_color)
print(pred_size)
print(pred_bg)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 

In [9]:
true_labels

[{'shape': ['circle',
   'circle',
   'circle',
   'circle',
   'circle',
   'circle',
   'circle',
   'circle'],
  'x_position': tensor([0., 0., 0., 0., 0., 0., 0., 0.]),
  'y_position': tensor([0., 0., 0., 0., 0., 0., 0., 0.]),
  'color': tensor([[0.4000, 0.4000, 0.6000],
          [0.4000, 0.4000, 0.6000],
          [0.6000, 0.4000, 0.4000],
          [0.4000, 0.4000, 0.6000],
          [0.6000, 0.4000, 0.4000],
          [0.6000, 0.4000, 0.4000],
          [0.4000, 0.4000, 0.6000],
          [0.6000, 0.4000, 0.4000]]),
  'size': tensor([0.3500, 0.3500, 0.3500, 0.5500, 0.3500, 0.3500, 0.3500, 0.3500]),
  'bg_color': tensor([[0.6000, 0.6000, 0.6000],
          [0.4000, 0.4000, 0.4000],
          [0.4000, 0.4000, 0.4000],
          [0.4000, 0.4000, 0.4000],
          [0.4000, 0.4000, 0.4000],
          [0.4000, 0.4000, 0.4000],
          [0.6000, 0.6000, 0.6000],
          [0.4000, 0.4000, 0.4000]]),
  'class': tensor([6, 7, 3, 5, 3, 3, 6, 3])},
 {'shape': ['circle',
   'circle',
   '

In [11]:
true_shape = convert_labels_2(true_labels, "shape").to(device)
true_color = convert_labels_2(true_labels, "color").to(device)
true_size = convert_labels_2(true_labels, "size").to(device)
true_bg = convert_labels_2(true_labels, "bg_color").to(device)

In [12]:
true_bg

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')

In [13]:
true_shape

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')

In [14]:
true_color 

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')

In [15]:
true_size

tensor([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0], device='cuda:0')